In [ ]:
import numpy as np
from IPython.display import display
from datetime import datetime
from utils import close_notebook, AppWidgets, read_json

In [ ]:
# Global variables
CONFIG_VARIABLES = read_json("config.json")

In [ ]:
# Widgets
app_widgets = AppWidgets(
    "deposition", 
    CONFIG_VARIABLES["raw_materials_types"], 
    CONFIG_VARIABLES["process_sample_types"], 
    CONFIG_VARIABLES["samples_collection_openbis_path"]
)

In [ ]:
# Connect with openBIS
app_widgets.connect_openbis()

# Deposition

## Select experiment, sample, and instrument

In [ ]:
experiments = app_widgets.openbis_session.get_collections(type = "EXPERIMENT")
samples = app_widgets.openbis_session.get_objects(type = "SAMPLE")
instruments = app_widgets.openbis_session.get_objects(type = "INSTRUMENT")
molecules = app_widgets.openbis_session.get_objects(type = "MOLECULE")

experiments_names_permids = [(f"{experiment.props['$name']} ({experiment.attrs.identifier})", experiment.permId) for experiment in experiments]
experiments_dates = [datetime.fromisoformat(experiment.registrationDate) for experiment in experiments]
experiments_names_permids = [x for _,x in sorted(zip(experiments_dates, experiments_names_permids), reverse = True)]
experiments_names_permids.insert(0, ('Select an experiment...', -1))
app_widgets.experiments_dropdown.options = experiments_names_permids

samples_names_permids = [(sample.props['$name'], sample.permId) for sample in samples]
samples_dates = [datetime.fromisoformat(sample.registrationDate) for sample in samples]
samples_names_permids = [x for _,x in sorted(zip(samples_dates, samples_names_permids), reverse = True)]
samples_names_permids.insert(0, ('Select an input sample...', -1))
app_widgets.samples_dropdown.options = samples_names_permids

instruments_names_permids = [(f"{instrument.props['$name']} ({instrument.attrs.permId})", instrument.permId) for instrument in instruments]
instruments_names_permids.insert(0, ('Select an instrument...', -1))
app_widgets.instruments_dropdown.options = instruments_names_permids
app_widgets.instruments_dropdown.options = instruments_names_permids

molecules_names_permids = [(f"{molecule.props['$name']} ({molecule.attrs.permId})", molecule.permId) for molecule in molecules]
molecules_empa_numbers = [np.nan if molecule.props['empa_number'] is None else int(molecule.props['empa_number']) for molecule in molecules]
molecules_batches = [np.nan if molecule.props['batch'] is None else molecule.props['batch'] for molecule in molecules]
molecules_names_permids = [x for _, _, x in sorted(zip(molecules_empa_numbers, molecules_batches, molecules_names_permids), key = lambda item: (item[0], item[1]), reverse=True)]
molecules_names_permids.insert(0, ('Select a molecule...', -1))
app_widgets.molecules_dropdown.options = molecules_names_permids
app_widgets.molecules_dropdown.options = molecules_names_permids

display(app_widgets.experiments_dropdown, app_widgets.sample_metadata_boxes, app_widgets.instruments_dropdown, app_widgets.molecule_metadata_boxes)
app_widgets.create_button.on_click(app_widgets.create_process_action)
app_widgets.quit_button.on_click(close_notebook)
app_widgets.samples_dropdown.observe(app_widgets.load_sample_metadata, names = 'value')
app_widgets.molecules_dropdown.observe(app_widgets.load_molecule_metadata, names = 'value')

## Properties

In [ ]:
display(app_widgets.method_name_textbox, app_widgets.method_properties, app_widgets.comments_textbox)

## Support Files

In [ ]:
display(app_widgets.support_files_uploader)

## Sample out

In [ ]:
app_widgets.method_name_textbox.observe(app_widgets.update_text, names = 'value')
display(app_widgets.sample_out_name_textbox)
display(app_widgets.bottom_buttons_hbox)
display(app_widgets.increase_buttons_size)